https://github.com/changhoonhahn/SEDflow/blob/spectra/bin/sedflow_p/data/training_sed.py

### script to generate training SED data for the SEDflow project

In [1]:
import os,sys
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from multiprocessing.pool import Pool 

# --- provabgs --- 
os.environ["SPS_HOME"] = "/media/SSD/Doktori/Csillagtomegbecsles_cikk/SED/SEDflow/fsps"
from provabgs import infer as Infer
from provabgs import models as Models

#### Input

In [2]:
Nsample = 125729
seed    = 901
n_cpu   = 8
version = '0.9' 
dat_dir = '/media/SSD/Doktori/Csillagtomegbecsles_cikk/SED/SEDflow/my_data/'

#### SPS parameter priors

In [3]:
prior_sps = Infer.load_priors([
    Infer.UniformPrior(7., 12.5, label='sed'),
    Infer.FlatDirichletPrior(4, label='sed'),           # flat dirichilet priors
    Infer.UniformPrior(0., 1., label='sed'),            # burst fraction
    Infer.UniformPrior(1e-2, 13.27, label='sed'),       # tburst
    Infer.LogUniformPrior(4.5e-5, 1.5e-2, label='sed'), # log uniform priors on ZH coeff
    Infer.LogUniformPrior(4.5e-5, 1.5e-2, label='sed'), # log uniform priors on ZH coeff
    Infer.UniformPrior(0., 3., label='sed'),            # uniform priors on dust1
    Infer.UniformPrior(0., 3., label='sed'),            # uniform priors on dust2
    Infer.UniformPrior(-2., 1., label='sed')            # uniform priors on dust_index
])

#### SPS model

In [4]:
m_sps = Models.NMF(burst=True, emulator=True)

input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index


#### Draw (parameters, z) from the prior

In [5]:
np.random.seed(seed)

thetas_sps  = np.array([prior_sps.transform(prior_sps.sample()) for i in range(Nsample)])
thetas_unt  = prior_sps.untransform(thetas_sps)
redshifts   = np.random.uniform(0, 0.06, Nsample) 
thetas      = np.concatenate([thetas_sps, np.atleast_2d(redshifts).T], axis=1)

#### Generate SED for (parameters, z) values 

In [6]:
def SED(tt): 
    w, f = m_sps.sed(tt[:-1], tt[-1]) 
    return [w, f]

with Pool(n_cpu) as p: 
    wfps = p.map(SED, thetas)

waves   = np.array([wfp[0] for wfp in wfps])
seds    = np.array([wfp[1] for wfp in wfps]) 

#### Calculating magnitudes, fluxes and other properties:

In [7]:
sys.path.append('/media/SSD/Doktori/Csillagtomegbecsles_cikk/SED/SEDflow/SEDflow/src/')
from sedflow import train as Train

def SED_to_mags(wave, sed, filters=None): 
    import astropy.units as U
    _sed = np.atleast_2d(sed) * 1e-17*U.erg/U.s/U.cm**2/U.Angstrom
    _wave = wave * U.Angstrom
    return filters.get_ab_magnitudes(_sed, wavelength=_wave)

mags = [SED_to_mags(waves[i], seds[i], filters = Train.photometry_bands() )for i in range(Nsample)]
mags = ( [[ mags[n][0][i] for i in range( 0, 5 )] for n in range(0, Nsample)] )
fluxes = 10**(0.4 * (22.5 - np.array(mags)) ) 
props = Train.thetas2props( thetas_sps, redshifts )

input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index


#### Saving results

In [8]:
np.save( os.path.join( dat_dir, 'train.v%s.thetas_sps.npy' % ( version ) ), thetas_sps )
np.save( os.path.join( dat_dir, 'train.v%s.thetas_unt.npy'  % ( version )  ), thetas_unt )
np.save( os.path.join( dat_dir, 'train.v%s.redshifts.npy'  % ( version )  ), redshifts )
np.save( os.path.join( dat_dir, 'train.v%s.waves.npy'  % ( version ) ), waves )
np.save( os.path.join( dat_dir, 'train.v%s.seds.npy'  % ( version ) ), seds )
np.save( os.path.join( dat_dir, 'train.v%s.mags_nonoise.npy' % ( version ) ), mags )
np.save( os.path.join( dat_dir, 'train.v%s.flux_nonoise.npy' % ( version ) ), fluxes )
np.save( os.path.join( dat_dir, 'train.v%s.props.npy' % ( version ) ), props )